In [32]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import keras.utils
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import OneHotEncoder

import numpy as np 
import pandas as pd 
import random
import sys
import io

In [4]:
song = pd.read_csv('entre_dos_aguas.csv')
# display(song)

,Note,Duration,Offset
0,0,384,0
1,56,72,384
2,0,24,456
3,57,72,480
4,0,24,552
5,56,72,576
6,0,24,648
7,57,72,672
8,0,18,744
9,5,6,762


In [5]:
notes = list(song['Note'])
notes = notes*5
durations = list(song['Duration'])
durations = durations*5

In [6]:
mx_notes = max(notes)+1
mx_dur = max(durations)+1
how_much = 50

In [7]:
seq_len = 20

X_notes = []
Y_notes = [] 

for i in range(0, len(notes)-seq_len, 1):
    seq_in = notes[i:i+seq_len]
    seq_out = notes[i + seq_len]
    
    X_notes.append(seq_in)
    Y_notes.append(seq_out)



In [8]:
X_dur = []
Y_dur = []

for i in range(0, len(durations)-seq_len-1):
    seq_in = durations[i:i+seq_len]
    seq_out = durations[i + seq_len]
    
    X_dur.append(seq_in)
    Y_dur.append(seq_out)

In [9]:
train_note = keras.utils.np_utils.to_categorical(X_notes)
test_note = keras.utils.np_utils.to_categorical(Y_notes)

In [10]:
train_dur = keras.utils.np_utils.to_categorical(X_dur)
test_dur =  keras.utils.np_utils.to_categorical(Y_dur)

In [11]:
print(train_note.shape)
print(test_note.shape)

(8010, 20, 67)
(8010, 67)


In [12]:
print(train_dur.shape)
print(test_dur.shape)

(8009, 20, 385)
(8009, 385)


In [13]:
model = Sequential()
model.add(LSTM(128, input_shape=(seq_len, mx_notes)))
model.add(Dense(mx_notes, activation='softmax'))
optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [14]:
dur_model = Sequential()
dur_model.add(LSTM(128, input_shape=(seq_len, mx_dur)))
dur_model.add(Dense(mx_dur, activation='softmax'))
dur_optimizer = RMSprop(learning_rate=0.01)
dur_model.compile(loss='categorical_crossentropy', optimizer=dur_optimizer)

In [46]:
note_gen = []

def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    start_index = random.randint(0, len(notes) - seq_len - 1)
    for diversity in [0.4 for i in range(5)]:
        print('----- diversity:', diversity)

        generated = []
        sentence = notes[start_index: start_index + seq_len]
        generated += sentence
        print('----- Generating with seed: ' + str(sentence))

        for i in range(how_much):
            x_pred = np.zeros((1, seq_len, mx_notes))
            for t, nt in enumerate(sentence):
                x_pred[0, t, nt] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)

            sentence = sentence[1:]
            sentence.append(next_index)
            note_gen.append(next_index)
    print(note_gen)


In [47]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(train_note, test_note,
          batch_size=128,
          epochs=1,
          callbacks=[print_callback])

Epoch 1/1
8010/8010 [==============================] - 3s 373us/step - loss: 0.8736

----- Generating text after Epoch: 0
----- diversity: 0.4
----- Generating with seed: [25, 25, 22, 21, 13, 21, 22, 25, 26, 0, 21, 21, 22, 0, 64, 0, 13, 44, 21, 0]
----- diversity: 0.4
----- Generating with seed: [25, 25, 22, 21, 13, 21, 22, 25, 26, 0, 21, 21, 22, 0, 64, 0, 13, 44, 21, 0]
----- diversity: 0.4
----- Generating with seed: [25, 25, 22, 21, 13, 21, 22, 25, 26, 0, 21, 21, 22, 0, 64, 0, 13, 44, 21, 0]
----- diversity: 0.4
----- Generating with seed: [25, 25, 22, 21, 13, 21, 22, 25, 26, 0, 21, 21, 22, 0, 64, 0, 13, 44, 21, 0]
----- diversity: 0.4
----- Generating with seed: [25, 25, 22, 21, 13, 21, 22, 25, 26, 0, 21, 21, 22, 0, 64, 0, 13, 44, 21, 0]
[65, 0, 21, 45, 22, 21, 45, 22, 21, 45, 22, 21, 45, 22, 21, 45, 22, 21, 45, 22, 0, 21, 21, 22, 25, 25, 22, 21, 13, 12, 11, 10, 7, 0, 10, 7, 7, 10, 7, 10, 14, 10, 14, 10, 7, 10, 14, 10, 10, 10, 65, 0, 21, 45, 22, 21, 45, 22, 21, 45, 22, 21, 45, 22, 

In [48]:
dur_gen = []


def dur_on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    start_index = random.randint(0, len(durations) - seq_len - 1)
    for diversity in [0.6 for i in range(5)]:
        print('----- diversity:', diversity)

        generated = []
        sentence = durations[start_index: start_index + seq_len]
        generated += sentence
        print('----- Generating with seed: ' + str(sentence))

        for i in range(how_much):
            x_pred = np.zeros((1, seq_len, mx_dur))
            for t, nt in enumerate(sentence):
                x_pred[0, t, nt] = 1.

            preds = dur_model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)

            sentence = sentence[1:]
            sentence.append(next_index)
            dur_gen.append(next_index)
    print(dur_gen)

In [49]:
dur_print_callback = LambdaCallback(on_epoch_end=dur_on_epoch_end)

dur_model.fit(train_dur, test_dur,
          batch_size=128,
          epochs=1,
          callbacks=[dur_print_callback])

Epoch 1/1
8009/8009 [==============================] - 4s 548us/step - loss: 0.6522

----- Generating text after Epoch: 0
----- diversity: 0.6
----- Generating with seed: [6, 6, 6, 6, 9, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 12, 12, 6]
----- diversity: 0.6
----- Generating with seed: [6, 6, 6, 6, 9, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 12, 12, 6]
----- diversity: 0.6
----- Generating with seed: [6, 6, 6, 6, 9, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 12, 12, 6]
----- diversity: 0.6
----- Generating with seed: [6, 6, 6, 6, 9, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 12, 12, 6]
----- diversity: 0.6
----- Generating with seed: [6, 6, 6, 6, 9, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 12, 12, 6]
[12, 6, 12, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 12, 12, 12, 12, 6, 12, 12, 6, 6, 6, 12, 12, 12, 6, 12, 12, 12, 12, 6, 6, 12, 6, 6, 6, 6, 6, 6, 6, 12, 6, 6, 12, 6, 6, 6, 6, 6, 6, 12, 12, 6, 6, 6, 6, 6, 6, 6, 6, 6,

In [52]:
off_gen = [] 
accum = 0 

for i in range(len(dur_gen)): 
    off_gen.append(accum)
    accum += dur_gen[i]
    
music_gen = pd.DataFrame(list(zip(note_gen, dur_gen, off_gen)), columns =['Note', 'Duration', 'Offset'])


music_gen.to_csv('generated_music.csv', sep=',', index=False)
